In [ ]:
import pandas as pd
import numpy as np

import stapes

In [ ]:
example_tri = pd.read_csv("examples/example_triangles.csv")
example_tri["DevLag"] = example_tri["DevLagId"]
example_train_tri = example_tri[example_tri["CalendarId"] <= 10]
example_test_tri = example_tri[example_tri["CalendarId"] > 10]

In [ ]:
example_train_tri

In [ ]:
train_data = {
    (name, int(row["TriangleId"]), int(row["ExpPeriodId"]), int(row["DevLagId"])): row[name]
    for name in ["ReportedLoss", "PaidLoss", "EarnedPremium", "DevLag"]
    for _, row in example_train_tri.iterrows()
    if (not np.isnan(row[name])) and int(row["TriangleId"]) == 3
}

In [ ]:
test_data = {
    (name, int(row["TriangleId"]), int(row["ExpPeriodId"]), int(row["DevLagId"])): row[name]
    for name in ["EarnedPremium", "DevLag"]
    for _, row in example_test_tri.iterrows()
    if (not np.isnan(row[name])) and int(row["TriangleId"]) == 3
}
test_coords = [
    ("ReportedLoss", tri_id, exp_id, dev_id)
    for name, tri_id, exp_id, dev_id in test_data
    if name == "EarnedPremium"
]

In [ ]:
model = stapes.build_model("""
pos :ata = vector(group=DevLagId);

ReportedLoss mean = ReportedLoss.prev_dev * :ata;
ReportedLoss variance = exp(:sigma_intercept + log(DevLag) * :sigma_slope) * ReportedLoss.prev_dev;
""")

In [ ]:
config = {
    "ReportedLoss__family": "lognormal",
    "ata__loc": 0.0,
    "ata__scale": 3.0,
    "ReportedLoss__missing_loc": 0.0,
    "ReportedLoss__missing_scale": 2.0,
}

In [ ]:
model.fit(train_data, config)

In [ ]:
np.mean(model.parameters["ata"].samples[".."][:, 1:], axis=0)

In [ ]:
predictions = model.predict(test_coords, test_data)